# Graph recommendation

In [ ]:
%load_ext kedro.extras.extensions.ipython

In [ ]:
%reload_kedro

In [ ]:
from typing import Iterator, Tuple
import re
import pickle
import _pickle as cPickle

from kedro.extras.datasets.pandas import CSVDataSet
from kedro.io.core import get_filepath_str
import numpy as np
import pandas as pd
import dgl
from dgl.sampling import sample_neighbors, select_topk
from dgl import load_graphs, save_graphs, DGLHeteroGraph
from dgl.data.heterograph_serialize import HeteroGraphData
from typing import Any, Dict, List, Tuple
import torch

import recommender_gnn.pipelines.santander_preprocessing.nodes
from recommender_gnn.pipelines.graph_recommendation_modeling.nodes import generate_graph_dgsr
from recommender_gnn.extras.datasets.chunks_dataset import (
 _concat_chunks,
)

pd.options.mode.chained_assignment = None
pd.set_option('expand_frame_repr', True)
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
dataset = "santander"

In [ ]:
transactions_graph_path = f"{dataset}.transactions_graph"
transactions_mapped_path = f"{dataset}_transactions_mapped"
users_mapping_path = f"{dataset}_users_mapping"
items_mapping_path = f"{dataset}_items_mapping"
neg_transactions_path = f"{dataset}_dgsr_negative_transactions_samples"

In [ ]:
transactions_mapped = _concat_chunks(context.catalog.load(transactions_mapped_path))
neg_transactions = context.catalog.load(neg_transactions_path)

In [ ]:
from ast import literal_eval

In [ ]:
neg_transactions[0]

# DGL graph loading

In [ ]:
def save_graphs_python(save_filepath: str, graph: dgl.DGLGraph, graph_dict: Dict) -> None:
    """Save heterographs into file using only Python functions instead of dgl C implementation"""
    if graph_dict is None:
        graph_dict = {}
    if isinstance(graph, DGLHeteroGraph):
        graph = [graph]
        graph_dict = [graph_dict]
    assert all(
        [type(g) == DGLHeteroGraph for g in graph]
    ), "Invalid DGLHeteroGraph in graph argument"
    gdata_list = [
        [g, graph_dict[i]] for i, g in enumerate(graph)
    ]
    with open(save_filepath, "wb") as file:
        pickle.dump(gdata_list, file)

In [ ]:
graph_path = "/home/michal/projects/gid-ml-framework/data/05_model_input/santander/santander_dgsr_kaggle_grm_predict_graphs/20580/20580_2.bin"

In [ ]:
with open(graph_path, 'rb') as f:
    gdata_list = pickle.load(f)

In [ ]:
gdata_list[0][1]

In [ ]:
gdata_list = load_graphs(graph_path)

In [ ]:
graph = gdata_list[0][0]
graph_dict = gdata_list[1]

In [ ]:
type(gdata_list[0][0]) == DGLHeteroGraph

In [ ]:
save_filepath = "/home/michal/projects/data_temp/4_1.bin"

In [ ]:
save_graphs_python(save_filepath, graph, graph_dict)

In [ ]:
with open(save_filepath, "wb") as file:
        pickle.dump(gdata_list, file)

# Saving testing fixtures graphs

In [ ]:
def mapped_transactions_custom():
    transactions_dict = {
        "user_id": {
            "0": 0,
            "1": 0,
            "2": 0,
            "3": 0,
            "4": 0,
            "5": 1,
            "6": 1,
            "7": 1,
            "8": 1,
            "9": 1,
            "10": 2,
            "11": 2,
            "12": 2,
            "13": 2,
            "14": 2,
            "15": 3,
            "16": 3,
            "17": 3,
            "18": 3,
            "19": 3,
            "20": 4,
        },
        "item_id": {
            "0": 0,
            "1": 1,
            "2": 2,
            "3": 2,
            "4": 3,
            "5": 0,
            "6": 1,
            "7": 2,
            "8": 5,
            "9": 3,
            "10": 0,
            "11": 1,
            "12": 2,
            "13": 4,
            "14": 3,
            "15": 7,
            "16": 1,
            "17": 2,
            "18": 6,
            "19": 3,
            "20": 8,
        },
        "time": {
            "0": 1453939200,
            "1": 1453039200,
            "2": 1453032200,
            "3": 1453132200,
            "4": 1453132200,
            "5": 1453939201,
            "6": 1453039202,
            "7": 1453032203,
            "8": 1453132204,
            "9": 1453132205,
            "10": 1453939206,
            "11": 1453039207,
            "12": 1453032208,
            "13": 1453132209,
            "14": 1453132210,
            "15": 1453939211,
            "16": 1453039212,
            "17": 1453032213,
            "18": 1453132214,
            "19": 1453132215,
            "20": 1453939216,
        },
    }
    transactions_df = pd.DataFrame(transactions_dict)
    return transactions_df

In [ ]:
transactions_custom = mapped_transactions_custom()

In [ ]:
graph_custom = generate_graph_dgsr(transactions_custom)

In [ ]:
graph_path = "../src/tests/fixtures/graphs/graph_custom.pkl"

In [ ]:
with open(graph_path, 'wb') as f:
    pickle.dump(graph_custom, f, protocol=-1)